**Tutorial 3 - Measuring a Spectral Line by Bayesian Parameter Estimation**

In this tutorial you will measure the strength and width of a spectral line 
using a Bayesian method.


1) Read in the data from the file tut_03_data.csv.  There are three 
columns: wavelength, flux and sigma.  Sigma is the known standard deviation 
of the flux in each pixel.  Plot the spectrum using matplotlib.pyplot.errorbar().

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pa

df = pa.read_csv('tut_03_data.csv')
.
.
.
plt.show()

2) First we need to subtract off the continuum. We know that there is a line centred around $\lambda \simeq 23$.  Take the part of the spectrum with $\lambda > 45$ which should just be background.  Assuming the noise is Gaussian and uncorrelated.   Using a uniform prior on the background.

In [ ]:

import scipy.optimize as opt

## Separate out the background dominated region of the spectrum, 
##  wavelength > 45,  Make vectors of the flux, wavelength and 
## sigma for these pixels.

df_bk_pixels = 

## Make a function that takes the background level b and 
## returns the log-likelihood calculated from the data 
## in the vectors above.  The pixels are statistically independent.
## You can use the df_bk_pixels dataframe made above inside the function
## you define.

def loglike(b):
    .
    .
    .

## Make an array of possible background values.  Call 
## it "background".
## To find the right range it is sometimes useful to 
## plot the log-likelihood over different ranges until 
## you find the maximum and you are sure the likelihood 
## is zero at both edges of the range.

db = 0.001 ## resolution in background
background = np.arange(-0.1,0.15,db)

## Calculate the likelihood at each of these background values
## using the function you have made.

likelihood = np.empty(len(background))
for i,b in enumerate(background) :
    .
    .
    .

## Normalize this numerically by summing the 
## likelihood array to get the posterior distribution 
## for the background.
## You should look at the plot and make sure that
## the sum is a good approximation of the integral, i.e. 
## the spacing in the parameter is small enough to make the 
## curve smooth and the range is large enough.
   
## plot posterior for background normalized numerically

plt.xlabel('background')
plt.ylabel('probability')
plt.title('posterior')
plt.show()

print 'Posterior at edges of parameter space: ',posterior[0],posterior[-1]

## Find maximum of posterior and the variance numerically.  
## Use opt.fmin()
## The integrals can be made into sums.
.
.
.
print 'The maximum likelihood background is :',bmax
print 'The average of posterior  :',bave
print 'The standard deviation of the posterior  :',np.sqrt(variance)


3) Now lets fit the line.  Assume the line has the profile

$ f(\lambda) = A \exp\left[- \left( \frac{(\lambda - \lambda_o)}{\Delta\lambda }\right)^2 \right] $

We want to find the parameters $A$, $\lambda_o$, $\Delta\lambda$ and the background.  

In [ ]:
.
.
.

## Write a function for the line profile that takes 
## the wavelength, A, center of line, lo, and the width 
## of the line dl.
def line_profile(l,A,lo,dl):
    .
    .
    .

## Write a function that takes the parameters 
## A,lo,dl,b and returns the log-likelihood for 
## the data.  The function should have the signature 
## def loglike(p): where p[0]=A, p[1]=lo, p[2]=dl and p[3]=b
## You should use the original dataframe for the data with 
## the full range of wavelength.
def loglike(p):
    .
    .
    .
    
## Find the maximum likelihood values for the parameters using 
## the whole data set.  Use the library function 
## scipy.optimize.minimize() to do this. Note that what  
## this function returns is a structure with more than just the solution.
## Read the documentation for this function before starting.  You will
## need to make a guess to start the minimization.  Make these 
## guesses from the plot of the data.  
.
.
.

print('maximum likelihood solution : ',maxlikelihood)

In [ ]:
## 4) Plot the best fit model over the plot of the data made in 
## part 1).  Use your line_profile() function.


plt.xlabel(r'$\lambda$')
plt.ylabel('flux')
plt.show()

5) Set all the parameters to their maximum likelihood values except the line strength $A$.  Make a plot of the **conditional posterior** for $A$ near its maximum likelihood value with uniform and Jaffreys priors on $A$.

In [ ]:

Aarray = ...

posterior = np.empty(len(Aarray))
for i,A in enumerate(Aarray):
    .
    .

## normalize the posterior
.
.
.

plt.plot(Aarray,posterior,label='uniform prior')
plt.plot(Aarray,posterior,label='Jeffreys prior')

plt.legend()
plt.xlabel('A')
plt.show()

## Doee the choice of prior change the result significantly?

6) Make a 2D map of the conditional posterior as a function of $A$ and $\Delta \lambda$ at the maximum likelihood values of the other parameters.  Do this by making a grid of $A$ and $\Delta \lambda$ values and using matplotlib.pyplot.contour().  Put the proper labels on the axes.

In [ ]:

dlarray = .....
Aarray = ...

.
.
.

## this is very useful for making 2D plots. X and Y are 2D arrays.
X, Y = np.meshgrid(dlarray, Aarray)

.
.
.

plt.xlabel('line width')
plt.ylabel('line strength')
plt.title("conditional posterior")

plt.show()

7) Write a function that takes values for $A$ and $\Delta \lambda$ and returns the 
posterior **marginalized** over $\lambda_o$.  Use scipy.integrate.quad() to do the 
integration.  You will need to make a new "posterior" function with the correct 
order of input parameters to use this function.  Use the maximum likelihood value for the background from part 2).  You might want to reduce the resolution of the grid to reduce the run time.

In [ ]:
from scipy.integrate import quad

# this reorders the parameters
def posterior(lo,A,dl,b):
    return np.exp(-loglike(np.array([A,lo,dl,b])))
    #def loglike(A,lo,dl,b):

def marginal_posterior(A,dl):
    ## use quad to integrate over lo only using the arg option
    .
    .
    .

.
.
.

plt.xlabel('line width')
plt.ylabel('line strength')
plt.title("marginal posterior")

plt.show()

8) Sum the 2D array from 7) to find the 1D marginalized posteriors for both 
$\Delta\lambda$ and $A$.  This is a numerical marginalization which can be done by simply summing along each of the axes.  The sum approximates an integral over the parameter.

Plot both marginal distributions.  If these are not smooth, you haven't used enough points in parameter space.

In [ ]:
#####
.
.
.
.

9) Find the mean and variance for $\Delta\lambda$ and the $A$ using the map of the marginalized posterior found above, the X and Y arrays from above, and the numpy.sum() function.  Each of these will be marginalized over all other variables except the background.

In [ ]:
.
.
.
.

In [ ]:
#########################################
#  This function finds the level for a contour that contains 
#  a fixed fraction of the total sum of pixels (or voxels).
#  Understand it. Run this box so that it is defined
#########################################
def find_level(posterior,fraction) :
    tot = np.sum(posterior)
    max = np.max(posterior)
    min = np.min(posterior)

    ## initialize level to halfway between max and min
    level = 0.5*(max + min) 
    ## initialize fraction for this level
    frac = np.sum( posterior[ posterior >= level ]  )/tot
    ## initialize resolution = +/- smallest pixel as fraction of total
    res = np.min( posterior[ posterior >= level ]  )/tot

    ## iterate until frac is within res of the input fraction
    while( abs(frac - fraction) > res  ) :
        
        ## update max or min
        if( frac > fraction) :
            min = level
        else :
            max = level
        
        ## update level by bisecting 
        level = 0.5*(max + min)
        
        ## update frac and res    
        frac = np.sum( posterior[ posterior >= level ] )/tot
        res = np.min( posterior[ posterior >= level ]  )/tot

    ## output the level and its actual fraction
    return level,frac

###

10) Using the function above, find the 68% and 95% confidence regions for $A$ and $\Delta\lambda$ and make a 2D contour plot of these levels.